### [Keras Tuner]

In [1]:
# 텐서플로를 위한 최적의 파라미터 세트를 선택하는데 도움을 주는 라이브러리

In [2]:
# 기계학습 (ML) 응용 프로그램에 대한 파라미터의 오른쪽 집합을 선택하는 과정은
# hyperparameter 조정 또는 hypertuning 이라고 함

In [3]:
# 하이퍼 파라미터는 학습 프로세스와 ML 모델의 토폴로지를 제어하는 변수

# 이러한 변수는 훈련 프로세스 도안 일정하게 유지되며
# ML 프로그램의 성능에 직접적인 영향을 미침

In [4]:
# 하이퍼 파라미터에는 2가지 유형이 있음

# 1. 모델 하이퍼 파라미터
# 2. 알고리즘 하이퍼 파라미터

#### < 설정 >

In [5]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [6]:
# Keras Tuner를 설치하고 가져오기

In [7]:
! pip install -q -U keras-tuner

In [8]:
import kerastuner as kt

#### < 데이터셋 다운로드 >

In [9]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [10]:
# 0~1 사이의 값으로 normalization

In [11]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

#### < 모델 정의 >

In [12]:
# 하이퍼 튜닝을 위한 모델일 빌드 할 때, 
# 모델 아키텍쳐 외에 하이퍼 파라미터 검색 공간도 정의

In [13]:
# 두 가지 접근방식을 통해 하이퍼 모델을 정의할 수 있음

# - 모델 빌더 기능을 사용
# - 서브 클래싱으로 HperModel Keras 튜너 API 클래스 사용

In [14]:
# 모델 작성기 함수를 사용하여 이미지 분류 모델을 정의해 보기

# 모델 작성기 함수는 컴파일 된 모델을 반환하고
# 인라인으로 정의한 하이퍼 파라미터를 사용하여 모델을 하이퍼 튜닝함

In [15]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # 첫번째 증의 유닛 숫자를 tune(조정) 
  # 32 ~ 512 사이의 옵션 값을 선택
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # 옵티마이저를 위해 학습률을 조정
  # 0.01, 0.001, 0.0001옵션 값을 선택
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [16]:
# 튜너를 인스턴스화 하고 튜닝을 수행

In [17]:
# Hyperband Tuner에는 네 가지 튜더가 있으며 각각 아래와 같음
# RandomSearch , Hyperband , BayesianOptimization, Sklearn 

In [18]:
# 하이퍼 밴드 튜너를 인스턴스화하려면 하이퍼 모델, objective, 
# max_epochs (max_epochs)를 지정해야 함

In [19]:
# 하이퍼밴드 튜너를 생성
tuner = kt.Hyperband(model_builder,objective = 'val_accuracy', max_epochs = 10,
                     factor = 3,directory = 'my_dir', project_name = 'intro_to_kt')

In [20]:
# 하이퍼 파라미터 검색을 실행하기 전에 
# 모든 학습 단계가 끝날 때 학습 출력을 지우는 콜백을 정의

In [21]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [22]:
# 초 매개변수 검색을 실해
tuner.search(img_train, label_train, epochs = 10, 
             validation_data = (img_test, label_test), 
             callbacks = [ClearTrainingOutput()])

Trial 30 Complete [00h 01m 02s]
val_accuracy: 0.883899986743927

Best val_accuracy So Far: 0.8841999769210815
Total elapsed time: 00h 08m 48s
INFO:tensorflow:Oracle triggered exit


In [24]:
# 하이퍼 파리미터를 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [25]:
# 최적의 파라미터로 모델을 다시 훈련시키기
model = tuner.hypermodel.build(best_hps)

In [26]:
model.fit(img_train, label_train, epochs = 10, 
          validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4893 - accuracy: 0.8261 - val_loss: 0.4200 - val_accuracy: 0.8525
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3662 - accuracy: 0.8666 - val_loss: 0.3801 - val_accuracy: 0.8614
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3275 - accuracy: 0.8790 - val_loss: 0.3682 - val_accuracy: 0.8618
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3031 - accuracy: 0.8880 - val_loss: 0.3672 - val_accuracy: 0.8691
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2877 - accuracy: 0.8935 - val_loss: 0.3613 - val_accuracy: 0.8714
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2735 - accuracy: 0.8979 - val_loss: 0.3304 - val_accuracy: 0.8793
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2588 - accuracy: 0.9030 - val_loss: 0.3331 - val_accuracy:

In [27]:
# my_dir/intro_to_kt 디렉토리에는 하이퍼 파라미터 검색 중에 실행 된 
# 모든 시험에 대한 자세한 로드와 체크 포인트가 포함되어 있음

In [28]:
# 하이퍼 파라미터 검색을 다시 실행하면 keras tuner는 
# 이러한 로그의 기존 상태를 사용하여 검색을 재개 함

# 이 동작을 비활성화 하려면 인스턴스화 과정에서 overwrite=True 인수를 전달